In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import itertools

In [ ]:
tools_dir_path = os.path.abspath('')
root_dir_path = os.path.dirname(tools_dir_path)
print(root_dir_path)
results_dir_path = os.path.join(root_dir_path, 'results')

In [ ]:
# cur_data_dir_name = 'data-3990x'
# cur_data_dir_name = 'm1-max-202207'
cur_data_dir_name = 'E3-1270v6'
# cur_data_dir_name = 'tempdata'
# cur_data_dir_name = 'data-xeon-6144'
# cur_data_dir_name = 'm1-max-202207'
# cur_data_dir_name = 'pi4'
cur_data_dir_path = os.path.join(results_dir_path, cur_data_dir_name)
cur_csv_dir_path = os.path.join(cur_data_dir_path, 'csv')
export_fig_dir_path = os.path.join(cur_data_dir_path, 'figs')
export_chartjs_dir_path = os.path.join(cur_data_dir_path, 'chart_js')
if not os.path.exists(export_fig_dir_path):
    os.mkdir(export_fig_dir_path)
if not os.path.exists(export_chartjs_dir_path):
    os.mkdir(export_chartjs_dir_path)

In [ ]:
filename_list = [f for f in os.listdir(cur_csv_dir_path) if os.path.isfile(os.path.join(cur_csv_dir_path, f))]
data_filename_pattern = re.compile(r'(.+)__(.+)__(.+)__(.+)\.csv')
data_filename_list = []
data_filepath_list = []
map_name_list = []
hash_name_list = []
key_name_list = []
value_name_list = []
k_v_dict = {}
for filename in filename_list:
    match_ret = data_filename_pattern.findall(filename)
    if match_ret:
        data_file_path = os.path.join(cur_csv_dir_path, filename)
        data_filepath_list.append(data_file_path)
        data_filename_list.append(filename)
        temp_map_name = match_ret[0][0]
        temp_hash_name = match_ret[0][1]
        map_name_list.append(temp_map_name)
        hash_name_list.append(temp_hash_name)
        key_name_list.append(match_ret[0][2])
        value_name_list.append(match_ret[0][3])
        temp_k_v_name =  '<' + match_ret[0][2] + ',' + match_ret[0][3] + '>'
        # k_v_name_list.append(temp_k_v_name)
        if not temp_k_v_name in k_v_dict:
            k_v_dict[temp_k_v_name] = {'data_filepath_list':[], 'hash_name_list':[], 'map_name_list':[]}
        k_v_dict[temp_k_v_name]['data_filepath_list'].append(data_file_path)
        k_v_dict[temp_k_v_name]['hash_name_list'].append(temp_hash_name)
        k_v_dict[temp_k_v_name]['map_name_list'].append(temp_map_name)
print("total %d data files" % (len(data_filename_list)))
map_name_set = set(map_name_list)
hash_name_set = set(hash_name_list)
print(data_filename_list)
print(map_name_set)
print(hash_name_set)
print(k_v_dict.keys())

In [ ]:
def get_color(index):
    color_table = [
        
        "#16a085","#27ae60","#2980b9","#8e44ad","#2c3e50",
        "#f1c40f","#e67e22","#e74c3c",
         "#d63031","#feca57","#5f27cd","#54a0ff","#01a3a4",
        "#25CCF7","#FD7272","#54a0ff","#00d2d3",
        "#1abc9c","#2ecc71","#3498db","#9b59b6","#34495e",
        # "#ecf0f1", "#bdc3c7", "#dfe6e9", 
        "#00b894","#00cec9","#0984e3","#6c5ce7","#ffeaa7",
        "#fab1a0","#ff7675","#fd79a8","#fdcb6e","#e17055",
        "#95a5a6","#f39c12","#d35400","#c0392b",
        "#55efc4","#81ecec","#74b9ff","#a29bfe",
        "#7f8c8d", # grey
        
       
    ]
    color_num = len(color_table)
    index %= color_num
    color_hex = color_table[index]
    color_int = int(color_hex.lstrip('#'),16)
    blue_int = color_int % 256
    color_int //= 256
    green_int = color_int % 256
    red_int = color_int // 256
    return red_int, green_int, blue_int

In [ ]:
def get_bg_color(red, green, blue):
    return (red + 255) // 2, (green + 255) // 2, (blue + 255) // 2

In [ ]:
def is_two_list_same(list1, list2):
    if len(list1) != len(list2):
        return False
    length = len(list1)
    for i in range(length):
        if list1[i] != list2[i]:
            return False
    return True

def encode_title(title):
    title = title.replace('<', "lb_")
    title = title.replace('>', "_rb_")
    title = title.replace(',', "_co_")
    title = title.replace('%', 'percent')
    return title

def decode_title(title):
    # use '{' instead of '<' because '<' cause some strange bugs when rendering
    title = title.replace('lb_', r'{')
    title = title.replace('_rb_', r'}')
    title = title.replace('_co_', ',')
    title = title.replace('percent', '%')
    return title

def get_chart_js_str(title, x_arr_list, y_arr_list, label_list, y_axis_label, ymax=None):
    ret_str = ''
    ret_str += "\n\n<!--start chart.js code for " + title + "-->\n"
    width = 400
    height = 350
    title = encode_title(title)
    ret_str += "\n<!--start canvas for " + title + "-->\n"
    ret_str += "<canvas id=\"%s_chart\" width=\"%d\" height=\"%d\"></canvas>" % (title, width, height)
    ret_str += "\n<!--end canvas for " + title + "-->\n\n"
    global_callback_arr_name = "create_chart_funcs"
    function_start_str = "function %s_create() {\n" % (title)
    function_end_str = "};\n"
    add_to_callback_arr_str = "%s.push(%s_create);\n"  % (global_callback_arr_name, title)
    arr_num = len(x_arr_list)
    for i in range(1, arr_num):
        assert(is_two_list_same(x_arr_list[0], x_arr_list[i]))
    x_labels_str = "const %s_labels = " % (title)
    element_arr = x_arr_list[0]
    x_labels_str += str(list(element_arr)) + ";\n"
    point_styles = itertools.cycle(('circle', 'rect', 'triangle', 'rectRot', 'rectRounded', 'cross', 'crossRot', 'star'))
    datasets_str = "datasets: [\n"
    point_radius = 6
    point_hover_radius = 12
    for i in range(arr_num):
        line_r, line_g, line_b = get_color(i)
        bg_r, bg_g, bg_b = get_bg_color(line_r, line_g, line_b)
        hidden_flag = 'false'
        if ymax is not None:
            if np.max(y_arr_list[i]) >= ymax:
                hidden_flag = 'true'
        dataset_str = "{\nlabel: '%s',\nbackgroundColor: 'rgba(%d,%d,%d,0.5)',\nborderColor: 'rgb(%d,%d,%d)',\npointStyle: '%s',\npointRadius:%d,\npointHoverRadius:%d,\nhidden: %s,\ndata: " % (
            label_list[i], bg_r, bg_g, bg_b, line_r, line_g, line_b, next(point_styles), point_radius, point_hover_radius, hidden_flag
        )
        dataset_str += str(list(y_arr_list[i])) + ",\n},\n"
        datasets_str += dataset_str
    datasets_str += "],\n"
    data_config_str = "const %s_data = {\nlabels: %s_labels,\n%s};\n" % (title, title, datasets_str)
    title_str = "const %s_title = {\ntext: '%s',\ndisplay: true,\n};\n" % (title, decode_title(title))
    scales_str = "const %s_scales = {\nx: {display: true, type: 'logarithmic',title: {text : 'element num', display: true,},},y: {display: true,type: 'logarithmic',title: {text : '%s',display: true,},}};" % (title, y_axis_label)
    options_str = "const %s_options = {\nscales: %s_scales,\nplugins:{\ntitle: %s_title,\nlegend: {\nlabels: {\nusePointStyle: true,},\n},\n},\n};" % (title, title, title)
    config_str = "const %s_config = {\n type: 'line',\ndata: %s_data,\noptions: %s_options,\n};\n" % (title, title, title)
    new_chart_str = "new Chart( document.getElementById('%s_chart'), %s_config,);" % (title, title)
    ret_str += "<script>\n\n" + function_start_str + x_labels_str + data_config_str + title_str + scales_str + options_str + config_str + new_chart_str + function_end_str + "\n\n</script>\n"
    ret_str += "\n\n<!-- Add to call back script --><script>\n\n" + add_to_callback_arr_str  + "\n\n</script>\n"
    ret_str += "<!--end chart.js code for " + title + "-->\n\n"
    return ret_str

In [ ]:
import matplotlib.ticker as plticker
from matplotlib.ticker import FormatStrFormatter
def plot_lines(title, x_arr_list, y_arr_list, label_list, y_axis_label, ymax=None, save_fig=False, show_fig=False, export_fig_dir_path=None, save_chart_js=False, export_chartjs_dir_path=None, save_name=None):
    # fix some typos of the existing data
    title = title.replace('_hit_hit_', '_hit_')
    fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
    fig.suptitle(title, fontsize='small')
    assert(len(x_arr_list) == len(y_arr_list))
    arr_num = len(x_arr_list)
    marker = itertools.cycle(('v','o','^','<','>','s','p','*','+','x','D')) 
    for i in range(arr_num):
        plot_num = 0
        for j in range(len(y_arr_list[i])):
            if y_arr_list[i][j] == 0.0:
                break
            plot_num += 1
        ax.plot(x_arr_list[i][:plot_num], y_arr_list[i][:plot_num], label=label_list[i], marker=next(marker), markersize=2.5, linewidth=1.0)
    ax.set_xlabel('element num')
    ax.set_ylabel(y_axis_label)
    ax.set_yscale('log')
    ax.set_xscale('log')
    default_ymin, default_ymax = ax.get_ylim()
    if ymax is None:
        # set latency for lookup latency if not set
        if ('lookup' in title) and ('latency in title'):
            ymax = 800
    if not ymax is None:
        if ymax < default_ymax:
            ax.set_ylim([default_ymin, ymax])
    subs = [1.0, 2.0, 4.0, 6.0]
    ax.yaxis.set_minor_locator(plticker.LogLocator(subs=subs))
    ax.yaxis.set_minor_formatter(FormatStrFormatter("%.0f"))
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    fig.legend(loc='center right', fontsize='xx-small')
    
    if save_fig:
        export_fig_name = title + '.png'
        if not save_name is None:
            export_fig_name = save_name
        export_fig_path = os.path.join(export_fig_dir_path, export_fig_name)
        fig.savefig(export_fig_path, transparent=False, facecolor='white')
    # if you need to show figures in the notebook, set show_fig as true
    if show_fig:
        plt.show()
    plt.close(fig)
    if save_chart_js:
        export_chart_js_filename = title + ".txt"
        export_chart_js_path = os.path.join(export_chartjs_dir_path, export_chart_js_filename)
        with open(export_chart_js_path, "w") as out_chart_js_f: 
            chart_js_str = get_chart_js_str(title, x_arr_list, y_arr_list, label_list, y_axis_label, ymax=ymax)
            out_chart_js_f.write(chart_js_str)
    # if out_fp is not None:
    #     chart_js_str = get_chart_js_str(title, x_arr_list, y_arr_list, label_list, y_axis_label, ymax=ymax)
    #     out_fp.write(chart_js_str)
    

In [ ]:
# whether show the figure in notebook
show_figure = False
# whether save the figure to picture files
save_figure = True
save_chart_js = True

# chart_js_fp = None
# if save_chart_js:
#     save_chart_js_path = os.path.join(export_chartjs_dir_path, "chart_js.html")
#     chart_js_fp = open(save_chart_js_path, mode="w")
for k_v_name, data_dict in k_v_dict.items():
    # print(k_v_name)
    data_file_num = len(data_dict['data_filepath_list'])
    map_hash_name_list = []
    element_num_arr_list = []
    construct_time_with_reserve_arr_list = []
    construct_time_without_reserve_arr_list = []
    erase_insert_arr_list = []
    hit_without_rehash_lookup_arr_list = []
    miss_without_rehash_lookup_arr_list = []
    may_without_rehash_lookup_arr_list = []
    hit_with_rehash_lookup_arr_list = []
    miss_with_rehash_lookup_arr_list = []
    may_with_rehash_lookup_arr_list = []
    iterate_arr_list = []
    with_final_rehash_construct_arr_list = []
    latency_arr_list_dict = {}
    for t in range(data_file_num):
        temp_map_hash_name = data_dict['map_name_list'][t] + "," + data_dict['hash_name_list'][t]
        map_hash_name_list.append(temp_map_hash_name)
        temp_data_filepath = data_dict['data_filepath_list'][t]
        temp_data_df = pd.read_csv(temp_data_filepath, sep=',')
        temp_data_df = temp_data_df.rename(columns=lambda x: x.strip())
        column_name_list = temp_data_df.columns.to_list()
        latency_item_pattern = re.compile(r'.+_latency')
        for column_name in column_name_list:
            match_obj = latency_item_pattern.match(column_name)
            if match_obj is not None:
                if column_name not in latency_arr_list_dict:
                    latency_arr_list_dict[column_name] = []
                latency_arr_list_dict[column_name].append(temp_data_df[column_name])
        element_num_arr_list.append(temp_data_df['element_num'])
        construct_time_with_reserve_arr_list.append(temp_data_df['avg_construct_time_with_reserve_ns'])
        construct_time_without_reserve_arr_list.append(temp_data_df['avg_construct_time_without_reserve_ns'])
        erase_insert_arr_list.append(temp_data_df['avg_erase_insert_ns'])
        hit_without_rehash_lookup_arr_list.append(temp_data_df['avg_hit_default_load_factor_lookup_ns'])
        miss_without_rehash_lookup_arr_list.append(temp_data_df['avg_miss_default_load_factor_lookup_ns'])
        may_without_rehash_lookup_arr_list.append(temp_data_df['avg_50%_hit_default_load_factor_lookup_ns'])
        hit_with_rehash_lookup_arr_list.append(temp_data_df['avg_hit_large_max_load_factor_lookup_ns'])
        miss_with_rehash_lookup_arr_list.append(temp_data_df['avg_miss_large_max_load_factor_lookup_ns'])
        may_with_rehash_lookup_arr_list.append(temp_data_df['avg_50%_hit_large_max_load_factor_lookup_ns'])
        iterate_arr_list.append(temp_data_df['avg_iterate_ns'])
        with_final_rehash_construct_arr_list.append(temp_data_df['avg_with_final_rehash_construct_ns'])
    plot_lines(k_v_name + '__avg_insert_time_with_reserve', element_num_arr_list, construct_time_with_reserve_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + '__avg_insert_time_without_reserve', element_num_arr_list, construct_time_without_reserve_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name +'__avg_erase_insert_time', element_num_arr_list, erase_insert_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + '__avg_hit_find_default_load_factor', element_num_arr_list, hit_without_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + '__avg_miss_find_default_load_factor', element_num_arr_list, miss_without_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + r'__avg_50%_hit_default_load_factor', element_num_arr_list, may_without_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + '__avg_hit_find_large_max_load_factor', element_num_arr_list, hit_with_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + '__avg_miss_find_large_max_load_factor', element_num_arr_list, miss_with_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + r'__avg_50%_hit_large_max_load_factor', element_num_arr_list, may_with_rehash_lookup_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, ymax=200, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + '__avg_iterate', element_num_arr_list, iterate_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    plot_lines(k_v_name + '__avg_construct_time_with_final_rehash', element_num_arr_list, with_final_rehash_construct_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
    for latency_item, data_arr_list in latency_arr_list_dict.items():
        plot_lines(k_v_name + ',' + latency_item, element_num_arr_list, data_arr_list, map_hash_name_list, 'ns', save_fig=save_figure, export_fig_dir_path=export_fig_dir_path, show_fig=show_figure, save_chart_js=save_chart_js, export_chartjs_dir_path=export_chartjs_dir_path)
# if save_chart_js:
#     chart_js_fp.close()